In [ ]:
import pandas as pd
from os import path, environ, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

In [ ]:
load_dotenv()

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [ ]:
input_dir = path.join('dados')

In [ ]:
filename = path.join(input_dir, 'distritos_original.csv')
df_distritos = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_distritos

In [ ]:
filename = path.join(input_dir, 'subprefeituras_original.csv')
df_subs = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_subs

In [ ]:
filename = path.join(input_dir, 'censo_demografia_original.csv')
df_pop = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype={'CD_DIST': str})
df_pop

In [ ]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs_qlik = pd.read_csv(url_subs)
df_subs_qlik

In [ ]:
df_subs_qlik = df_subs_qlik[['sub.CODIGO', 'sub.NOME']]
df_subs_qlik

# Transformação dos dados

## Unindo as tabelas de distritos e subprefeituras

Vemos que as subprefeituras possuem duas colunas diferentes de id: `cd_identificador_subprefeitura` e `cd_subprefeitura`. Como as os valores nas duas colunas não coincidem, vamos avaliar qual delas é a mais adequada para fazermos a junção com os dados dos distritos.

In [ ]:
df_subs.sort_values('cd_identificador_subprefeitura').head(5)

In [ ]:
df_distritos.sort_values('cd_identificador_subprefeitura').head(5)

Vemos que ocorrem duas coisas: a coluna `cd_identificador_subprefeitura` da tabela de distritos coincide com o valor numérico da coluna `cd_subprefeitura`, mas a coluna `cd_subprefeitura` está em formato de texto, iniciando com zeros à esquerda. Portanto, vamos adicionar à tabela de distritos uma nova coluna `cd_subprefeitura`, convertendo o valor da coluna `cd_identificador_subprefeitura` para texto e preenchendo com zeros à esquerda para que a string tenha 2 caracteres.

In [ ]:
df_distritos.insert(
    3,
    'cd_subprefeitura',
    df_distritos['cd_identificador_subprefeitura'].astype(str).str.zfill(2)
)
df_distritos.head()

Agora, vamos unir as tabelas de distritos e subprefeituras, utilizando a coluna `cd_subprefeitura` como chave de junção.

In [ ]:
df_dist_sub = (
    df_distritos.drop(columns=['cd_identificador_subprefeitura'])
    .merge(
        df_subs.drop(columns=['cd_identificador_subprefeitura']),
        on='cd_subprefeitura',
        how='left'
    )
)

df_dist_sub.head()

Investigando o resultado no data wrangler, vemos que a junção ocorreu corretamente, com todos os distritos associados à sua respectiva subprefeitura.

## Agregando e unindo dados da população

Agora, precisamos preparar os dados da população para que possamos associá-los às subprefeituras. Vamos agrgar as colunas de população em dois grupos: população de 0 a 69 anos e população total.

In [ ]:
df_pop_summary = df_pop[['CD_DIST', 'NM_DIST']].copy()
df_pop_summary

In [ ]:
df_pop_summary.insert(
    2,
    'populacao_total',
    df_pop.iloc[:, 2:].sum(axis=1)
)

df_pop_summary

In [ ]:
df_pop_summary.insert(
    2,
    'populacao_0_69',
    df_pop_summary['populacao_total'] - df_pop['V01041']
)

df_pop_summary

## Unindo os dados de população por distritos

Vamos criar uma coluna com os nomes dos distritos sem acentuação e em letras maiúsculas para possibilitar a união dos dados.

In [ ]:
df_pop_summary.insert(
    2,
    'nm_distrito_municipal',
    df_pop_summary['NM_DIST'].apply(lambda x: unidecode(x).upper().strip())
)

df_pop_summary

Agora vamos ordenar alfabeticamente os dataframes e comparar os nomes dos distritos em cada tabela.

In [ ]:
{
    dist_censo : dist_gs
    for dist_censo, dist_gs in zip(
        sorted(df_pop_summary['nm_distrito_municipal'].unique()),
        sorted(df_dist_sub['nm_distrito_municipal'].unique())
    )
}

Todos os valores parecem corretos. Vamos ter certeza utilizando uma list comprehension para comparar os nomes dos distritos em cada tabela.

In [ ]:
any([dist_censo != dist_gs
    for dist_censo, dist_gs in zip(
        sorted(df_pop_summary['nm_distrito_municipal'].unique()),
        sorted(df_dist_sub['nm_distrito_municipal'].unique())
    )])

Todos os valores coincidem, então podemos fazer a junção dos dataframes utilizando a coluna `nm_distrito` como chave de junção.

In [ ]:
df_pop_summary = (
    df_pop_summary
    .merge(
        df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
        left_on='nm_distrito_municipal',
        right_on='nm_distrito_municipal',
        how='left'
    )
)

df_pop_summary

## Agregando os dados por subprefeitura

Finalmente, basta agregar os dados por subprefeitura para obter a população total e a população de 0 a 69 anos em cada subprefeitura.

In [ ]:
df_pop_subs = (
    df_pop_summary
    .groupby(['cd_subprefeitura', 'nm_subprefeitura'], as_index=False)
    .agg({
        'populacao_total': 'sum',
        'populacao_0_69': 'sum'
    })
)
df_pop_subs

## Padronizando os nomes das subprefeituras

Agora, falta apenas padronizar os nomes das subprefeituras para o mesmo utilizado no Qlik Sense. Primeiro, vamos comparar os nomes das subprefeituras em cada tabela.

In [ ]:
{
    sub_pop : sub_qlik
    for sub_pop, sub_qlik in zip(
        sorted(df_pop_subs['nm_subprefeitura'].unique()),
        sorted(df_subs_qlik['sub.NOME'].unique())
    )
}

Os nomes aparentam coincidir, mas vamos ter certeza utilizando uma list comprehension para comparar os nomes das subprefeituras em cada tabela.

In [ ]:
any([
    sub_pop != sub_qlik
    for sub_pop, sub_qlik in zip(
        sorted(df_pop_subs['nm_subprefeitura'].unique()),
        sorted(df_subs_qlik['sub.NOME'].unique())
    )
])

De fato, os nomes estão todos iguais. Vamos conferir o código e, caso sejam iguais, apenas renomeamos as colunas para o padrão do Qlik Sense.

In [ ]:
(
    df_pop_subs[['cd_subprefeitura', 'nm_subprefeitura']]
    .merge(
        df_subs_qlik,
        left_on='nm_subprefeitura',
        right_on='sub.NOME',
        how='left'
    )
    .sort_values('cd_subprefeitura')
)

Os números também coincidem, com a diferença de formato entre texto e número. Portanto, podemos finalizar o dataframe com os dados de população por subprefeitura, renomeando as colunas para o padrão do Qlik Sense e redefinindo os tipos.

In [ ]:
df_pop_subs = (
    df_pop_subs
    .rename(columns={
        'cd_subprefeitura': 'sub.CODIGO',
        'nm_subprefeitura': 'sub.NOME',
        'populacao_total': 'sub.POPULACAO_TOTAL',
        'populacao_0_69': 'sub.POPULACAO_0_69'
    })
    .astype({
        'sub.CODIGO': int,
        'sub.NOME': str,
        'sub.POPULACAO_TOTAL': int,
        'sub.POPULACAO_0_69': int
    })
)

df_pop_subs

## Checagem final dos dados

Para validar os dados, vamos checar se a soma total da população por subprefeitura é igual à população total da cidade de São Paulo.

In [ ]:
df_pop_subs['sub.POPULACAO_TOTAL'].sum()

In [ ]:
df_pop.iloc[:,2:].sum().sum()

In [ ]:
11451999-11434513

In [ ]:
1-11434513/11451999

O site do panorama do censo 2022 (https://censo2022.ibge.gov.br/panorama/) informa que a população total de São Paulo é de 11.451.999 habitantes, o que apresenta uma diferença de 17.486 habitantes em relação ao total agregado por subprefeitura (11.434.513 habitantes). Essa diferença representa apenas 0,15% da população total, mas, de todo modo, devemos voltar a investigar a origem dessa discrepância em etapas futuras do projeto.

Por ora, armazenaremos apenas a coluna de população de 0 a 69 anos, que é a que nos interessa para os dados de mortalidade.

# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [ ]:
base_path = path.join('data_output')

if not path.exists(base_path):
    makedirs(base_path)

filepath = path.join(base_path, f'subprefeitura_populacao_0_69.csv')

(
    df_pop_subs
    .drop(columns=['sub.POPULACAO_TOTAL'])
    .to_csv(filepath,
            index=False,
            sep=';',
            decimal=',',
            encoding='latin1')
)